In [1]:
%%capture
import waymax
import numpy as np
import mediapy
from tqdm import tqdm
import dataclasses
import pickle
import tensorflow as tf

from waymax import config as _config
from waymax import dataloader
from waymax import datatypes
from waymax import visualization

2024-10-07 06:46:01.719061: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-07 06:46:01.725681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 06:46:01.732972: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 06:46:01.735181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-07 06:46:01.741146: I tensorflow/core/platform/cpu_feature_guar

### Configurations

In [2]:
data_path = '/home/emerge/daphne/gpudrive/data/processed/validation_110'

### Get Waymo validation files

Steps

```bash
gcloud auth login
```

```
gsutil -m cp -r gs://waymo_open_dataset_motion_v_1_1_0/uncompressed/scenario/validation/ data/raw
```


In [3]:
from pathlib import Path

filenames = [
    p.name for p in Path(data_path).iterdir()
]

scenario_ids = [p.split("tfrecord-")[1].split(".json")[0] for p in filenames]

In [4]:
#scenario_ids

### Search through Waymax files, find a match and store the scenario

- Using the WOMD validation set from version 1.1.0. See [link](https://github.com/waymo-research/waymax/blob/720f9214a9bf79b3da7926497f0cd0468ca3e630/waymax/config.py#L269)

In [5]:
def _preprocess(serialized: bytes) -> dict[str, tf.Tensor]:
    womd_features = dataloader.womd_utils.get_features_description(
        include_sdc_paths=data_config.include_sdc_paths,
        max_num_rg_points=data_config.max_num_rg_points,
        num_paths=data_config.num_paths,
        num_points_per_path=data_config.num_points_per_path,
    )
    womd_features['scenario/id'] = tf.io.FixedLenFeature([1], tf.string)

    deserialized = tf.io.parse_example(serialized, womd_features)
    parsed_id = deserialized.pop('scenario/id')
    deserialized['scenario/id'] = tf.io.decode_raw(parsed_id, tf.uint8)

    return dataloader.preprocess_womd_example(
        deserialized,
        aggregate_timesteps=data_config.aggregate_timesteps,
        max_num_objects=data_config.max_num_objects,
    )

def _postprocess(example: dict[str, tf.Tensor]):
    scenario = dataloader.simulator_state_from_womd_dict(example)
    scenario_id = example['scenario/id']
    return scenario_id, scenario

In [6]:
data_config = dataclasses.replace(
    _config.WOD_1_1_0_VALIDATION, 
    max_num_objects=32
)

data_iter = dataloader.simulator_state_generator(config=data_config)

# Caution: This takes about a minute
scenario = next(data_iter)

I0000 00:00:1728297964.276972 1600125 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-07 06:46:04.294741: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [7]:
# Get a single scenario
scene_id_iter = dataloader.get_data_generator(
    data_config, _preprocess, _postprocess
)

In [8]:
# Get the unique ID string for a scenario
for scenario_id, scenario in scene_id_iter:
    # Decode bytes and return the scenario ID
    scenario_id = scenario_id.tobytes().decode('utf-8')

    if scenario_id in scenario_ids:
        print(f"Found scenario: {scenario_id}! Saving to file...")
        scenario_path = f'waymax_scenario_{scenario_id}.pkl'
        with open(scenario_path, 'wb') as f:    
            pickle.dump(scenario, f)
        break
    
else: 
    print('Done')

Found scenario: ab2a72c63f8fd589! Saving to file...


In [9]:
scenario_id

'ab2a72c63f8fd589'

In [10]:
'ab2a72c63f8fd589' in scenario_ids

True